In [ ]:
"""
Purpose: To download meshes from allen and
insert into segment and mesh table in database

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

In [3]:
import datajoint_utils as du

INFO - 2021-04-20 21:21:00,296 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 21:21:00,299 - settings - Setting database.user to celiib
INFO - 2021-04-20 21:21:00,299 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 21:21:00,304 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-20 21:21:00,305 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-20 21:21:00,319 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-20 21:21:00,742 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-20 21:21:00,746 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 21:21:00,747 - settings - Setting database.user to celiib
INFO - 2021-04-20 21:21:00,748 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 21:21:00,749 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 21:21:00,750 - settings - Setting database.user to celiib
INFO - 2021-04-20 21:21:00,752 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 21:21:00,753 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-20 21:21:01,212 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-04-20 21:21:01,373 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 21:21:01,375 - settings - Setting database.user to celiib
INFO - 2021-04-20 21:21:01,376 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 21:21:01,380 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-20 21:21:01,904 - settings - Setting enable_python_native_blobs to True


# Segment to Fetch

In [ ]:
two_soma_segment_ids_new = [
    
864691136672813959,
864691135472075826,
864691136311833405,
864691135564804951,
864691135758491982,
864691135945417764,
864691135644807919,
864691136008578990,
864691135510515408,
864691135815482447,
864691136236730639,
864691134988608634,
864691135382990571,
864691136617134811,
864691135753592525,
864691135841095907,
864691136056568536,
864691135609490439,
864691135926327508,
864691135474614848,
864691135341057989,
864691137196942401,
864691135939404545,
864691135155711844,
864691136008496556,
864691136008729518,
864691135883914352,
864691135345291423,
864691136041004886,
864691137196949569,
864691135975576131,
864691135463533125,
864691135653543364,
864691135373423816,
864691135850470343,
864691135210232512,
864691135775809453,
864691136486808850,
864691136008511148,
864691136031686843,
864691135576995845,
864691135490245479,
864691136008436140,
864691135571307557,
864691135753669837,
864691136090951092,
864691135065024068,
864691136922900452,
864691135807230237,
864691135373676873,
864691135782479155,
864691136922748900,
864691135850464967,
]

three_soma_segment_ids_new = []

In [ ]:
segment_ids = two_soma_segment_ids_new
len(segment_ids)

# Insert Segment IDs into a table that can control downstream analysis

In [ ]:
import datajoint as dj

In [ ]:
@schema
class AllenProofreading(dj.Manual):
    definition="""
    segment_id : bigint unsigned  #segment id for those to be decimated
    month: tinyint #the month that the cell was added
    day: tinyint #the day that the cell was added 
    year: int unsigned  #the year that the cell was added
    ---
    n_somas : int #the number of somas associated with the neuron from the Allen Institute
    
    """
month = 1
day = 28
year = 2021

double_somas =two_soma_segment_ids_new
triple_somas = three_soma_segment_ids_new
    
dict_of_seg_2_soma = [dict(segment_id=k,n_somas=2,month=month,day=day,year=year) for k in double_somas]
dict_of_seg_3_soma = [dict(segment_id=k,n_somas=3,month=month,day=day,year=year) for k in triple_somas]
AllenProofreading.insert(dict_of_seg_2_soma + dict_of_seg_3_soma,skip_duplicates=True)
AllenProofreading() & dict(month=1,day=28,year=2021)

In [ ]:
missing_segment_ids = (minnie.AllenProofreading() - minnie.Mesh()).fetch("segment_id")
missing_segment_ids

In [ ]:
du.download_and_insert_allen_meshes(missing_segment_ids,
                                   n_threads=10)

In [5]:
missing_segment_ids = (minnie.AutoProofreadValidationSegmentMap() - minnie.Mesh()).fetch("segment_id")
missing_segment_ids

array([864691136194248918, 864691135508879113, 864691135415666362,
       864691135975633475, 864691136812081779, 864691135358985048])

In [6]:
du.download_and_insert_allen_meshes(missing_segment_ids,
                                   n_threads=6)

Done in 122.420036 seconds.


In [7]:
print("hi")

hi


In [9]:
minnie.AutoProofreadValidationSegmentMap() - minnie.Decimation()

ver materialization version,nucleus_id nucleus id,segment_id the segment id in the proofread version,old_ver materialization version,old_segment_id the segment id in the proofread version--
49.00,296726,864691136194248918,0.08,864691135748575017
49.00,460391,864691135508879113,0.08,864691135272164113
49.00,518853,864691135415666362,0.08,864691135866394390
49.00,559081,864691135975633475,0.08,864691136333790899
49.00,581967,864691136812081779,0.08,864691136535575714
49.00,588839,864691135358985048,0.08,864691135699269154
